In [1]:
import pandas as pd
import os

In [2]:
## Read clean csv's
clean_hawaii_measurements_path = os.path.join('Resources', 'clean_hawaii_measurements.csv')
hawaii_measurements_df = pd.read_csv(clean_hawaii_measurements_path)
clean_hawaii_stations_path = os.path.join('Resources', 'clean_hawaii_stations.csv')
hawaii_stations_df = pd.read_csv(clean_hawaii_stations_path)


In [3]:
## Create database
from sqlalchemy import create_engine
engine = create_engine('sqlite:///hawaii.sqlite')

# Import and establish Base for which classes will be constructed
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float, ForeignKey



hawaii_measurements_df.reset_index(inplace=True)
hawaii_measurements_df.rename(columns={'index': 'id'}, inplace=True)

In [4]:
## Define the measurement class
class Measurement(Base):
    __tablename__ = 'measurement'
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(String)
    prcp = Column(Float)
    tobs = Column(Integer)


In [5]:
## Define the stations class
class Station(Base):
    __tablename__ = 'station'
    station = Column(String, ForeignKey('measurement.station'), primary_key=True)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)


In [8]:
## create tables
Base.metadata.create_all(engine)
## populate tables
hawaii_measurements_df.to_sql(name='measurement', con=engine, if_exists = 'replace', index=False)
hawaii_stations_df.to_sql(name='station', con=engine, if_exists = 'replace', index=False)
## commit changes
from sqlalchemy.orm import Session
session = Session(bind=engine)
session.commit()